In [8]:
import DataGen

In [49]:
importlib.reload(DataGen)

<module 'DataGen' from '/Users/sri/P/Steel/DataGen.py'>

In [9]:
import importlib

In [44]:
importlib.reload(data)

<module 'data' from '/Users/sri/P/Steel/data.py'>

In [50]:
ds = DataGen.DataGenerator(data.preproc_train_csv, img_dir=data.train_dir)

In [51]:
for t_img, t_mask in ds:
    assert False

AssertionError: 

In [52]:
t_img.shape

TensorShape([4, 256, 400, 3])

In [53]:
t_mask.shape

TensorShape([4, 256, 400, 4])

In [1]:
import data

In [2]:
import pandas as pd

In [3]:
import dataset

In [4]:
df = dataset.get_modified_df(data.train_csv)

100%|██████████| 50272/50272 [00:12<00:00, 4149.60it/s]


In [7]:
df.to_csv(data.root_dir + 'pre_proc_train.csv', index=False)

In [2]:
import pathlib

In [3]:
root_dir = '/Volumes/Transcend/Data/Steel/'
csv_file = root_dir + 'train.csv'
img_dir = root_dir + 'train/'
mask_dir = root_dir + 'mask/'

In [4]:
p = pathlib.Path(mask_dir)

In [5]:
import numpy as np

In [6]:
from PIL import Image

In [9]:
from tqdm import tqdm

In [10]:
m_max = -1
for mask in tqdm(p.glob('*.jpg')):
    img = Image.open(mask)
    img = np.array(img)
    mm = np.max(img)
    if mm > m_max:
        m_max = mm

12568it [01:40, 125.13it/s]


In [11]:
m_max

255

In [1]:
import dataset

In [2]:
import importlib

In [3]:
importlib.reload(dataset)

<module 'dataset' from '/Users/sri/P/Steel/dataset.py'>

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
import tensorflow as tf

In [7]:
from PIL import Image

In [23]:
root = '/Volumes/Transcend/Data/Steel/'

In [24]:
img = Image.open('/Volumes/Transcend/Data/Steel/mask/0a4ad45a5.jpg')

In [25]:
img = np.array(img)

In [26]:
np.max(img)

255

In [12]:
img_gen = dataset.gen_dataset()

100%|██████████| 50272/50272 [00:10<00:00, 4682.40it/s]


In [21]:
img_max = -1
mask_max = -1
count = 0
for elem in dataset.gen_dataset():
    count += 1
    img = elem[0]
    mask = elem[1]
    i_max = np.max(img)
    if i_max > img_max:
        img_max = i_max
    m_max = np.max(mask)
    if m_max > mask_max:
        mask_max = m_max

100%|██████████| 50272/50272 [00:09<00:00, 5086.44it/s]


In [22]:
count

2

In [19]:
mask_max

0.7764705882352941

In [10]:
assert False

AssertionError: 

In [ ]:
root_dir = '/Volumes/Transcend/Data/Steel/'
csv_file = root_dir + 'train.csv'
train_dir = root_dir + 'train/'
mask_dir = root_dir + 'mask/'

In [ ]:
img_dataset = dataset.get_dataset(csv_file, train_dir, mask_dir)

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
def gen_masks():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_mask = dataset.get_img_paths(df, mask_dir)
    for path in img_mask:
        img = Image.open(path)
        img = np.array(img)
        yield tf.convert_to_tensor(img)

In [ ]:
def gen_imgs():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_mask = dataset.get_img_paths(df, train_dir)
    for path in img_mask:
        img = Image.open(path)
        img = np.array(img)
        yield tf.convert_to_tensor(img)

In [ ]:
def gen_img_masks():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_masks = dataset.get_img_paths(df, mask_dir)
    img_paths = dataset.get_img_paths(df, train_dir)
    for img_path, img_mask in zip(img_paths, img_masks):
        img = Image.open(img_path)
        img = np.array(img)
        mask = Image.open(img_mask)
        mask = np.array(mask)
        yield tf.convert_to_tensor(img), tf.convert_to_tensor(mask)

In [ ]:
ds = tf.data.Dataset.from_generator(gen_img_masks, (tf.float32, tf.float32), (tf.TensorShape([256, 1600, 3]), tf.TensorShape([256, 1600, 4])))

In [ ]:
count = 0
for img, mask in ds:
    count += 1
    if count > 10:
        break
    print(img.shape)
    print(mask.shape)

In [ ]:
tf_dataset = dataset.get_dataset(csv_file, train_dir, mask_dir)

In [ ]:
df = dataset.get_modified_df(csv_file)

In [ ]:
all_img_paths = dataset.get_img_paths(df, train_dir)

In [ ]:
all_img_mask_path = dataset.get_img_paths(df, mask_dir)

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(all_img_paths)

In [ ]:
image_ds = image_ds.map(dataset.load_process_img)

In [ ]:
mask_ds = tf.data.Dataset.from_tensor_slices(all_img_mask_path)

In [ ]:
mask_ds = mask_ds.map(dataset.load_process_img)

In [ ]:
image_mask_ds = tf.data.Dataset.zip((image_ds, mask_ds))

In [ ]:
'/'.join(all_img_paths[9].split('/')[:-2])

In [ ]:
all_img_masks = dataset.get_img_rle(df)

In [ ]:
mask = dataset.get_img_masks(all_img_masks[9])

In [ ]:
img = Image.fromarray(mask, mode='CMYK')

In [ ]:
img.save(mask_dir + 'test.jpg')

In [ ]:
m_img = Image.open(mask_dir + 'test.jpg')

In [ ]:
import numpy as np

In [ ]:
m_np_img = np.array(m_img)

In [ ]:
np.array_equal(m_np_img, mask)

In [ ]:
def gen_masks(img_paths, img_masks, mask_dir):
    for path, rle in zip(img_paths, img_masks):
        mask = dataset.get_img_masks(rle)
        path = path.split('/')
        img_id = path[-1]
        mask_path = mask_dir + '/' + img_id
        mask_img = Image.fromarray(mask, mode='CMYK')
        mask_img.save(mask_path)

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(all_img_paths)

In [ ]:
image_df = image_ds.map(dataset.load_process_img)

In [ ]:
mask_ds = tf.data.Dataset.from_tensor_slices(all_img_masks)

In [ ]:
all_masks = [dataset.get_img_masks(mask) for mask in all_img_masks]

In [ ]:
mask_ds = mask_ds.map(tf.py_function(dataset.get_img_masks))